In [1]:
import os
import jax
import jax.numpy as jnp



In [2]:
key = jax.random.PRNGKey(0)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [10]:
import math as m
import numpy as np
import pandas as pd
import random



"""
生成100个长度为100的数据序列,每个数据序列包含两个长度为100的随机数序列（Seq1和Seq2）和一个目标值（Seqy）
"""
T = 750  # 时间序列长度
iterations = 1500  # 迭代次数(生成的样本数量)
Data = []
for i in range(iterations):
    Seq1 = []
    Seq2 = [0.00001 for _ in range(T)]
    Seqy = []
    # rand_index = random.sample(range(0, T), 2)    # 随机数作索引，即随机位置
    rand_index1 = random.choice(range(0, T //2))
    rand_index2 = random.choice(range(T // 2, T))
    for j in range(T):
        data_1 = random.random()  # 生成0-1的随机数
        Seq1.append(data_1)
    # Seq2[rand_index[0]] = 0.99999   # 序列②中的两个数 变成0.9999
    # Seq2[rand_index[1]] = 0.99999
    Seq2[rand_index1] = 0.99999   # 序列②中的两个数 变成0.9999
    Seq2[rand_index2] = 0.99999
    # Y_true = Seq1[rand_index[0]] + Seq1[rand_index[1]]  # 序列①中的两个数相加
    Y_true = Seq1[rand_index1] + Seq1[rand_index2]  # 序列①中的两个数相加
    Seqy.append(Y_true)
    Seq = Seq1+Seq2+Seqy
    Data.append(Seq)
Data = pd.DataFrame(Data)
# Data.to_csv('lp_100_1500.csv', header=None, index=None)

In [11]:
Data

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,0.541223,0.899890,0.154643,0.335773,0.672835,0.650906,0.866744,0.271312,0.436135,0.754204,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.455638
1,0.166354,0.520835,0.582120,0.066896,0.770840,0.338560,0.064402,0.771920,0.606023,0.564327,...,0.00001,0.00001,0.00001,0.99999,0.00001,0.00001,0.00001,0.00001,0.00001,1.027354
2,0.987226,0.060582,0.222062,0.672966,0.640120,0.845287,0.096188,0.699427,0.917066,0.620395,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.631944
3,0.009102,0.591258,0.773314,0.890506,0.913016,0.527603,0.730223,0.766992,0.478251,0.996184,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,1.006859
4,0.741567,0.703294,0.110723,0.537424,0.231361,0.607831,0.522833,0.316786,0.306939,0.593560,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,1.518128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.345727,0.918230,0.656405,0.021267,0.998376,0.895629,0.003740,0.298666,0.607942,0.190010,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.896905
1496,0.867828,0.177235,0.720012,0.542841,0.648331,0.244517,0.842636,0.264729,0.670242,0.819112,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.953492
1497,0.418629,0.794889,0.252557,0.410236,0.647417,0.314358,0.025527,0.812707,0.770828,0.575058,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,1.209448
1498,0.524503,0.076429,0.440002,0.105108,0.650388,0.259272,0.555666,0.669026,0.749746,0.271792,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,1.336818


In [4]:
def generate_dataset(features,labels,M_total,M_train,M_test,key=jax.random.PRNGKey(0)):
    
    # subsample train and test split
    test_indices = jax.random.choice(key,M_total, shape=(M_test,), replace=False)
    
    train_indices = jax.random.choice(key,
        jnp.setdiff1d(jnp.arange(M_total), test_indices), shape=(M_train,), replace=False)
        
    x_train, y_train = features[train_indices], labels[train_indices]
    x_test, y_test = features[test_indices], labels[test_indices]

    return jnp.array(x_train),jnp.array(y_train),jnp.array(x_test),jnp.array(y_test)

In [5]:
def save_dataset(n_qubits,n_layers,M_train,M_test,data_type,M_total,datasets_path):
    # Create directory if it doesn't exist
    os.makedirs(f"{datasets_path}/{data_type}", exist_ok=True)
    n_features =  n_qubits * n_layers * 3  # Dimensionality
    n_samples = M_total
    features,labels = generate_tanh_dataset(n_samples,n_features)
    print(features.shape)
    features = features.reshape(n_samples,n_layers,n_qubits,3)
    x_train,y_train,x_test,y_test = generate_dataset(features,labels,M_total,M_train,M_test)
    jnp.save(f"{datasets_path}/{data_type}/x_train_qubit_{n_qubits}_layer_{n_layers}_sample_{M_train}.npy",x_train)
    jnp.save(f"{datasets_path}/{data_type}/y_train_qubit_{n_qubits}_layer_{n_layers}_sample_{M_train}.npy",y_train)
    jnp.save(f"{datasets_path}/{data_type}/x_test_qubit_{n_qubits}_layer_{n_layers}_sample_{M_test}.npy",x_test)
    jnp.save(f"{datasets_path}/{data_type}/y_test_qubit_{n_qubits}_layer_{n_layers}_sample_{M_test}.npy",y_test)
    

In [10]:
M_train = 600
M_test = 10000
M_total = 11000
n_qubits = 2
data_type = "regression"
datasets_path = '../../datasets'

In [11]:
for l in range(1,11):
    save_dataset(n_qubits,l,M_train=M_train,M_test=M_test,M_total=M_total,data_type=data_type,datasets_path=datasets_path)

(11000, 6)
(11000, 12)
(11000, 18)
(11000, 24)
(11000, 30)
(11000, 36)
(11000, 42)
(11000, 48)
(11000, 54)
(11000, 60)
